<a href="https://colab.research.google.com/github/MengOonLee/LLM/blob/main/References/LangChain/ipynb/T10_Local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

pip install --no-cache-dir -qU \
    torch torchvision lightning transformers accelerate \
    langchain langgraph langchain-core langchain-community \
    langchain-huggingface

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import transformers
import time

model_name = "microsoft/Phi-4-mini-reasoning"
local_path = "./models"
os.makedirs(local_path, exist_ok=True)

start_time = time.time()
model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name)
model.save_pretrained(save_directory=local_path)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name)
tokenizer.save_pretrained(save_directory=local_path)
end_time = time.time() - start_time

print("Time taken: %.2f"%(end_time))

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=local_path)
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=local_path)

hf_pipeline = pipeline(model=model, tokenizer=tokenizer,
    task="text-generation")
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
from langchain_core.prompts import PromptTemplate
import time

template = """
Question: {question}
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is SWIFT messaging?"

start_time = time.time()
response = chain.invoke({"question": question})
end_time = time.time() - start_time

print("Time take: %.4f, %s"%(end_time, response))